# Secret Box

The `secretbox` API uses secret key authenticated encryption to encrypt and authenticate a message with a secret key that all parties must share.

[Libsodium Documentation](https://doc.libsodium.org/secret-key_cryptography/secretbox)

In [4]:
%load_ext sql
%sql postgresql://postgres@/

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Encryption requires a key and a nonce.  The nonce doesn't have to be confidential, but it should never ever be reused with the same key. The easiest way to generate a nonce is to use `crypto_secretbox_noncegen`:

In [ ]:
key = %sql select pgsodium.crypto_secretbox_keygen()::text
key = key[0][0]

nonce = %sql select pgsodium.crypto_secretbox_noncegen()::text
nonce = nonce[0][0]

## Encryption

A new secretbox is created with the key and the nonce:

In [18]:
secretbox = %sql SELECT crypto_secretbox::text from pgsodium.crypto_secretbox('bob is your uncle', :nonce, (:key)::bytea)
secretbox = secretbox[0][0]
print('The encrypted secretbox is: ', secretbox)

 * postgresql://postgres@/
1 rows affected.
 * postgresql://postgres@/
1 rows affected.
 * postgresql://postgres@/
1 rows affected.
The encrypted secretbox is:  \x7b11d8e3659f6fe2a7762f082019c607d5d64fd5f805f6ff6df68266664a6ec335


## Decryption

Decryption requires the same key and nonce.

In [19]:
plaintext = %sql SELECT crypto_secretbox_open FROM pgsodium.crypto_secretbox_open(:secretbox, :nonce, (:key)::bytea)
print('The decrypted message is :', plaintext[0][0].tobytes().decode('utf8'))

 * postgresql://postgres@/
1 rows affected.
The decrypted message is : bob is your uncle
